In [113]:
import pandas as pd
from datetime import datetime
start = datetime.now()
import os
import torch
import numpy as np
import argparse
import matplotlib.pyplot as plt

from torchvision import transforms
from tqdm import tqdm

from bindsnet         import encoding
from bindsnet.datasets import MNIST
from bindsnet.encoding import PoissonEncoder
from bindsnet.models import DiehlAndCook2015
from bindsnet.network.monitors import Monitor
from bindsnet.utils import get_square_assignments, get_square_weights
from bindsnet.evaluation import all_activity, proportion_weighting, assign_labels
from bindsnet.analysis.plotting import (
    plot_input,
    plot_assignments,
    plot_performance,
    plot_weights,
    plot_spikes,
    plot_voltages,
)
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline

In [114]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import TensorDataset
from torch.optim import Adam, SGD

from sklearn.decomposition import PCA
import pylab

In [115]:
import os
import torch
import numpy as np
import argparse
import matplotlib.pyplot as plt

from torchvision import transforms
from tqdm import tqdm

from bindsnet         import encoding
from bindsnet.datasets import MNIST
from bindsnet.encoding import PoissonEncoder
from bindsnet.models import DiehlAndCook2015
from bindsnet.network.monitors import Monitor
from bindsnet.utils import get_square_assignments, get_square_weights
from bindsnet.evaluation import all_activity, proportion_weighting, assign_labels
from bindsnet.analysis.plotting import (
    plot_input,
    plot_assignments,
    plot_performance,
    plot_weights,
    plot_spikes,
    plot_voltages,
)
from sklearn.model_selection import train_test_split
plot=False
gpu=False
seed = 0#args.seed
n_neurons = 100

n_clamp = 1
exc = 22.5
inh = 120
theta_plus = 0.05
time = 250
dt = 1.0
intensity = 32
progress_interval = 10
update_interval = 250      #250
train = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if gpu and torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
else:
    torch.manual_seed(seed)
    device = "cpu"
    if gpu:
        gpu = False

torch.set_num_threads(os.cpu_count() - 1)
print("Running on Device = ", device)

if not train:
    update_interval = n_test

n_classes = 2
n_sqrt = int(np.ceil(np.sqrt(n_neurons)))
start_intensity = intensity
per_class = int(n_neurons / n_classes)

# Build Diehl & Cook 2015 network.
network = DiehlAndCook2015(
    n_inpt=49282,
    n_neurons=n_neurons,
    exc=exc,
    inh=inh,
    dt=dt,
    nu=[1e-10, 1e-3],  # 0.711
    norm=4928.2,
    theta_plus=theta_plus,
    inpt_shape=(1, 1, 49282),
)

# Directs network to GPU
if gpu:
    network.to("cuda")

# Voltage recording for excitatory and inhibitory layers.
exc_voltage_monitor = Monitor(network.layers["Ae"], ["v"], time=time)
inh_voltage_monitor = Monitor(network.layers["Ai"], ["v"], time=time)
network.add_monitor(exc_voltage_monitor, name="exc_voltage")
network.add_monitor(inh_voltage_monitor, name="inh_voltage")


Running on Device =  cpu


In [116]:
# parameters:

run_version=54
normalization_multiplied=100
sampling_method='under'     #'under'  #over

plot=True
gpu=True
seed = 0#args.seed
n_neurons =100
n_clamp = 1
exc = 22.5
inh = 120
theta_plus = 0.05
time = 2000
dt = 1.0
intensity = 32
progress_interval = 50
update_interval = 150#250      #250
train = True

In [117]:
filler_i = 0.01
attacksize_j = 0.1
attack_model_k = 'random'
sampling_method='' 

%run ./Data_Preprocess.ipynb
x,y=dataset_movielens_daily(target_size_var=100,filler_size_var=filler_i,attack_size_var=attacksize_j,attack_model_var=attack_model_k)
y[['target']]=y[['target']].replace('1',1) 

if sampling_method=='under':
    undersample = RandomUnderSampler(sampling_strategy='majority')
    X_over, y_over = undersample.fit_resample(x, y)
    x=X_over
    y=y_over

if sampling_method=='over':
    oversample = RandomOverSampler(sampling_strategy='minority')
    X_over, y_over = oversample.fit_resample(x, y)
    x=X_over
    y=y_over  

if sampling_method=='SMOTE':
    over=SMOTE(sampling_strategy=0.25)
    under=RandomUnderSampler(sampling_strategy='majority')
    steps=[('o',over),('u',under)]
    pipeline=Pipeline(steps=steps)
    x,y=pipeline.fit_resample(x,y)
    
x.shape

(1682, 214)

In [118]:
## normalizing  
x=((x-x.min())/(x.max()-x.min()))*normalization_multiplied


In [119]:
##reshaping data to square:
for i in range(215,226):
    x[i]=0

## normalizing  
x=((x-x.min())/(x.max()-x.min()))*normalization_multiplied

x = x.values.reshape((x.shape[0],1,15,15))

x[np.isnan(x)] = 0



In [120]:
x.shape

(1682, 1, 15, 15)

In [121]:
## split train and test data
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.20, random_state=40)

x_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(Y_train['target'].values).float()

x_test_tensor = torch.tensor(X_test).float()
y_test_tensor = torch.tensor(Y_test['target'].values).float()

In [122]:
x_train_tensor.shape

torch.Size([1345, 1, 15, 15])

In [123]:
# Record spikes during the simulation.
spike_record = torch.zeros(update_interval, time, n_neurons, device=device)

# Neuron assignments and spike proportions.
assignments = -torch.ones_like(torch.Tensor(n_neurons), device=device)
proportions = torch.zeros_like(torch.Tensor(n_neurons, n_classes), device=device)
rates = torch.zeros_like(torch.Tensor(n_neurons, n_classes), device=device)

# Sequence of accuracy estimates.
accuracy = {"all": [], "proportion": []}

# Labels to determine neuron assignments and spike proportions and estimate accuracy
labels = torch.zeros(update_interval, device=device)
print(labels)

spikes = {}
for layer in set(network.layers):
    spikes[layer] = Monitor(network.layers[layer], state_vars=["s"], time=time)
    network.add_monitor(spikes[layer], name="%s_spikes" % layer)

# Train the network.
print("Begin training.\n")

inpt_axes = None
inpt_ims = None
spike_axes = None
spike_ims = None
weights_im = None
assigns_im = None
perf_ax = None
voltage_axes = None
voltage_ims = None
n_train = x_train_tensor.shape[0]
n_test = x_test_tensor.shape[0]
pbar = tqdm(total=n_train)


  0%|          | 0/1345 [00:00<?, ?it/s]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
Begin training.



In [129]:
from torch import nn
class convCIFAR(nn.Module):
    
    def __init__ (self):
        super().__init__()
        self.conv1 = nn.Conv1d(15,1,5)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))    
        return(x)

In [138]:
for i, (image,label) in enumerate(zip(x_train_tensor,y_train_tensor)):
#     print(image)
    input_2d_img=image
    cnn2d_1 = nn.Conv1d(in_channels=15, out_channels=1, kernel_size=5)
    a=nn.ReLU(cnn2d_1)
#     a=convCIFAR(image)
#     a=cnn2d_1(input_2d_img)
#     a=nn.ReLU()
    print(a)
#     print("cnn2d_1: \n")
#     print(cnn2d_1[0])
#     print(cnn2d_1(input_2d_img).shape, "\n")
#     print(cnn2d_1(input_2d_img))
#     print(image.shape,"\n")
#     print(image)

#     if i > n_train:
        
#         break
            
    encoded_image=encoding.poisson(datum=a, time=time, dt=dt)
#     encoded_label=encoding.poisson(datum=label, time=time)

#     image = encoded_image#datum["encoded_image"]
#     label = encoded_label#datum["label"]

ReLU(
  inplace=True
  (inplace): Conv1d(15, 1, kernel_size=(5,), stride=(1,))
)


TypeError: '>=' not supported between instances of 'ReLU' and 'int'

##### 

In [9]:
# for (i, datum) in enumerate(dataloader):
for i, (image,label) in enumerate(zip(x_train_tensor,y_train_tensor)):
    print ('i is: ', i )

    if i > n_train:
        break
    encoded_image=encoding.poisson(datum=image, time=time, dt=dt)
    encoded_label=encoding.poisson(datum=label, time=time)
    
    image = encoded_image#datum["encoded_image"]
    label = encoded_label#datum["label"]
    
#     image_arr = torch.sum(image)#image.numpy()
#     label_arr = torch.sum(label)#label.numpy()

#     result=result.append(pd.DataFrame({'image':[image_arr],'label':[label_arr],'i':[i]}))
#     result.to_csv('result4.csv')

    
# #     print('image is \n',image)
# #     print('label is \n',label)
#     image_arr = torch.sum(image)#image.numpy()
#     label_arr = torch.sum(label)#label.numpy()
    


    if i % update_interval == 0 and i > 0:
        print('went to if loop\n  i is',i , '\n update_interval is ', update_interval)
        # Get network predictions.
        all_activity_pred = all_activity(spike_record, assignments, n_classes)
        
        proportion_pred = proportion_weighting(
            spike_record, assignments, proportions, n_classes
        )

        # Compute network accuracy according to available classification strategies.
        accuracy["all"].append(
            100 * torch.sum(labels.long() == all_activity_pred).item() / update_interval
        )
        accuracy["proportion"].append(
            100 * torch.sum(labels.long() == proportion_pred).item() / update_interval
        )

        print(
            "\nAll activity accuracy: %.2f (last), %.2f (average), %.2f (best)"
            % (accuracy["all"][-1], np.mean(accuracy["all"]), np.max(accuracy["all"]))
        )
        print(
            "Proportion weighting accuracy: %.2f (last), %.2f (average), %.2f (best)\n"
            % (
                accuracy["proportion"][-1],
                np.mean(accuracy["proportion"]),
                np.max(accuracy["proportion"]),
            )
        )
        
        print('all_activity_pred is ', all_activity_pred)
        print('proportion_pred is ', proportion_pred)
        
        image_arr = torch.sum(image)#image.numpy()
        label_arr = torch.sum(label)#label.numpy()
        all_activity_pred_arr=torch.sum(all_activity_pred)#all_activity_pred.numpy()
#         torch.sum(outputs)
#         proportion_pred_arr=proportion_pred.numpy()
#         result=result.append(pd.DataFrame({'image':[image_arr],'label':[label_arr],'pred_all':[all_activity_pred],'propor':[proportion_pred]}))
        result=result.append(pd.DataFrame({'image':[image_arr],'label':[label_arr],'pred_all':[all_activity_pred_arr],'propor':[i]}))



        # Assign labels to excitatory layer neurons.
        assignments, proportions, rates = assign_labels(
            spike_record, labels, n_classes, rates
        )

    # Add the current label to the list of labels for this update_interval
    print('i % update interval is ',i % update_interval)
    print('label 0 is ', label[0])
    
    labels[i % update_interval] = label[0]

    # Run the network on the input.
    choice = np.random.choice(int(n_neurons / n_classes), size=n_clamp, replace=False)
    clamp = {"Ae": per_class * label.long() + torch.Tensor(choice).long()}
    if gpu:
        inputs = {"X": image.cuda().view(time, 1, 1, 1, 49282)}
    else:
        inputs = {"X": image.view(time, 1, 1, 1, 49282)}
    network.run(inputs=inputs, time=time, clamp=clamp)

    # Get voltage recording.
    exc_voltages = exc_voltage_monitor.get("v")
    inh_voltages = inh_voltage_monitor.get("v")

    # Add to spikes recording.
    spike_record[i % update_interval] = spikes["Ae"].get("s").view(time, n_neurons)

    # Optionally plot various simulation information.
    if plot:
        inpt = inputs["X"].view(time, 49282).sum(0).view(1, 49282)
        input_exc_weights = network.connections[("X", "Ae")].w
        square_weights = get_square_weights(
            input_exc_weights.view(49282, n_neurons), n_sqrt, 28
        )
        square_assignments = get_square_assignments(assignments, n_sqrt)
        voltages = {"Ae": exc_voltages, "Ai": inh_voltages}

        inpt_axes, inpt_ims = plot_input(
            image.sum(1).view(1, 49282), inpt, label=label, axes=inpt_axes, ims=inpt_ims
        )
        spike_ims, spike_axes = plot_spikes(
            {layer: spikes[layer].get("s").view(time, 1, -1) for layer in spikes},
            ims=spike_ims,
            axes=spike_axes,
        )
        weights_im = plot_weights(square_weights, im=weights_im)
        assigns_im = plot_assignments(square_assignments, im=assigns_im)
        perf_ax = plot_performance(accuracy, ax=perf_ax)
        voltage_ims, voltage_axes = plot_voltages(
            voltages, ims=voltage_ims, axes=voltage_axes
        )

        plt.pause(1e-8)


    network.reset_state_variables()  # Reset state variables.
    pbar.set_description_str("Train progress: ")
    pbar.update()

    

print("Progress: %d / %d \n" % (n_train, n_train))
print("Training complete.\n")
result.to_csv('result.csv')



In [ ]:
>>> torch.Size([16, 3, 28, 28])
    # 4d: [batch_size, channels, height, width]
    # use for nn.Conv2d() input.

In [14]:
def __init__(self, spatial_model, temporal_model):
    super(FusionNet, self).__init__()
    self.spatial_model = spatial_model
    self.temporal_model = temporal_model
    self.fc = nn.Sequential(nn.Linear(1024*7*2, 1024, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5),
                            nn.Linear(1024, 1))

def forward(self, spatial_input, temporal_input):
    spatial_output = self.spatial_model(spatial_input)
    temporal_output = self.temporal_model(temporal_input)
    fused = torch.cat((spatial_output, temporal_output), dim = 1)
    out = fused.view(fused.size(0), -1)
    out = self.fc(out)
    return out

In [ ]:
def ConvNet(self):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
#     self.layer2 = nn.Sequential(
#         nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
#         nn.ReLU(),
#         nn.MaxPool2d(kernel_size=2, stride=2))
    self.drop_out = nn.Dropout()
    self.fc1 = nn.Linear(7 * 7 * 64, 1000)
    self.fc2 = nn.Linear(1000, 10)

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

In [ ]:
n_users=X_train.shape[0]
n_days=X_train.shape[1]
n_items=X_train.shape[2]
#input_shape=(1,732,213,1682)
#input_shape=(213,)
timesteps = 1
data_dim=213
kernel_size=4
filters=32
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (1, 3), activation='relu'), input_shape =(n_days, 1, n_items, 1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(1,3))))
# model.add(TimeDistributed(Conv2D(32, (1, 3), activation='relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(1,3))))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(Flatten()))


X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1,  X_train.shape[2],1))